In [29]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [30]:
# !pip install tensorflow-gpu
# !pip install keras

In [31]:
# import keras
# config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1} ) 
# sess = tf.compat.v1.Session(config=config) 
# keras.backend.set_session(sess)

In [32]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import os
import sys
import tensorflow as tf

from tensorflow import keras
from tensorflow.python.keras import layers
from tensorflow.python.keras.layers import Input, Dense, Flatten, Dropout, Conv2D, GlobalMaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.optimizer_v2.adam import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers as opt
from sklearn.model_selection import train_test_split
import tensorflow.keras.layers as tfl

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import RandomRotation

In [33]:
import os
import shutil
import random
import glob
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [51]:
imgs = "../input/capstone1/Banknote1"
batch_size=32
img_size=224

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  imgs,
  class_names = ['20', '50', '100', '200', '500', '1000'],
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_size, img_size),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  imgs,
  class_names = ['20', '50', '100', '200', '500', '1000'],
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_size, img_size),
  batch_size=batch_size)

test_ds = tf.keras.preprocessing.image_dataset_from_directory('../input/testing/valid',
                                                  image_size=(img_size, img_size),
                                                  batch_size=batch_size)

num_classes = len(train_ds)

In [35]:
num_classes = len(train_ds)

In [36]:
categories = os.listdir(imgs)
categories = categories[:20]
print("List of categories =", categories, "\n\nNo. of categories =", len(categories))

In [37]:
class_names = train_ds.class_names
    
plt.figure(figsize=(10,10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [38]:
preprocess_input = tf.keras.applications.resnet50.preprocess_input

In [39]:
# AUTOTUNE = tf.data.experimental.AUTOTUNE
# train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)

In [40]:
image_batch, label_batch = next(iter(train_ds))

In [41]:
data_augmentation = Sequential(
  [
#     layers.RandomFlip("vertical",
#                       input_shape=(img_height,
#                                   img_width,
#                                   3)),
    keras.layers.RandomRotation(0.1),
    keras.layers.RandomZoom(0.1),
  ]
)

plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")


In [42]:
data_augmentation = keras.Sequential(
    [
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.2, interpolation='nearest'),
        tf.keras.layers.experimental.preprocessing.RandomContrast((0.2 ))
        
    ]
)

plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

In [43]:
train_ds = data_augmentation

In [44]:
#from tensorflow.keras.applications import ResNet50
#import keras
from keras.layers import Conv2D, DepthwiseConv2D, Dense, GlobalAveragePooling2D
#from keras.layers import Activation, BatchNormalization, Add, Multiply, Reshape
from keras.layers.core import Dense, Dropout, Flatten
from keras.models import Sequential,Model
from tensorflow.keras.applications.resnet50 import ResNet50

In [45]:
base_model = tf.keras.applications.ResNet50(input_shape=(224, 224, 3),
                                              include_top=False,
                                              weights= 'imagenet')

model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.GlobalMaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(512),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(6, activation='softmax')
])

In [46]:
base_model.trainable = False


In [47]:
model.summary()

In [48]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [52]:
steps_per_epoch_training = len(train_ds)/batch_size
steps_per_epoch_validation = len(val_ds)/batch_size
num_epochs = 10

In [53]:
fit_history = model.fit(train_ds, steps_per_epoch = steps_per_epoch_training, epochs = num_epochs,
                       validation_data=val_ds, validation_steps=steps_per_epoch_validation, verbose=1)

In [54]:
fig1 = plt.gcf()
plt.plot(fit_history.history['accuracy'])
plt.plot(fit_history.history['val_accuracy'])
plt.axis(ymin=0.4,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

In [55]:
model.save('BanknoteResnet50.h5')

In [57]:
ls BanknoteResnet50.h5

In [58]:
from tensorflow import keras
model = keras.models.load_model('BanknoteResnet50.h5')

In [59]:
tflite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = tflite_converter.convert()
open("BanknoteResNet50.tflite", "wb").write(tflite_model)